In [1]:
import pandas as pd

In [2]:
hce_bids = pd.read_csv('data/cash_flow_hce_bid.csv', index_col=0, parse_dates=True)

In [3]:
transformer_interval = pd.read_csv('data/cash_flow_transformer_interval.csv', index_col=0, parse_dates=True)

In [4]:
meter_interval = pd.read_csv('data/cash_flow_meter_interval.csv', index_col=0, parse_dates=True)

In [5]:
market_interval = pd.read_csv('data/cash_flow_market_interval.csv', index_col=0, parse_dates=True).iloc[:1]

In [6]:
start_times = list(market_interval["start_time"])
start_times

['2021-07-22T09:00:01']

In [7]:
p_clear = list(market_interval["p_clear"])
p_clear

[0.63]

### Sum all ((qmtp in TABLE meter_intervals for which p_bid <= p_clear))*p_clear

In [8]:
# meter_interval.loc[(meter_ineterval.p_bid <= )]
merge_meter_market_interval = meter_interval.merge(market_interval, on="start_time", how="left")

In [9]:
x = merge_meter_market_interval.loc[merge_meter_market_interval.p_bid <= merge_meter_market_interval.p_clear]

In [10]:
x["payments_to_pv_owner"] = x["p_clear"]*x["q_bid"]

/var/folders/tp/xzkt258d77dgg1hgjm1bkb6x_vqyb8/T/ipykernel_70523/2657359622.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["payments_to_pv_owner"] = x["p_clear"]*x["q_bid"]


In [11]:
payments_to_pv_owners = x.groupby("start_time")["payments_to_pv_owner"].sum()
payments_to_pv_owners

start_time
2021-07-22T09:00:01    0.24885
Name: payments_to_pv_owner, dtype: float64

###  (q from TABLE transformer_intervals at start_time == t) * (p_bid in TABLE hce_bids at start_time == t)
### Use p_bid for which is_supply = 1 if q >= 0
### Use p_bid for which is_supply = 0 if q < 0


Cash Flow:

Y-axis: at each t:

Select p_clear from market_intervals with start_time == t

Calculate payments to PV owners: Sum all ((qmtp in TABLE meter_intervals for which p_bid <= p_clear))*p_clear

Calculate import costs: (q from TABLE transformer_intervals at start_time == t) * (p_bid in TABLE hce_bids at start_time == t)

 - Use p_bid for which is_supply = 1 if q >= 0

 - Use p_bid for which is_supply = 0 if q < 0

Calculate retail income (inelastic load): (unresp_load from TABLE transformer_intervals at start_time == t) * fixed_retail_rate

THEN: retail income MINUS import costs MINUS payments to PV owners → THIS IS WHAT GETS PLOTTED

X-axis: time last 24 hours


In [12]:
merge_transformer_market_interval = transformer_interval.merge(market_interval, on="start_time", how="left")

In [13]:
merge_transformer_market_interval

,import_capacity,export_capacity,q,transformer_interval_id,start_time,end_time_x,unresp_load,transformer_id,market_interval_id,q_clear,p_exp,p_clear,p_dev,end_time_y,alpha,market_id
0,1072.480,536.240,0.000,1,2021-09-22T09:00:00,2021-09-22T09:05:00,0.00000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,992.420,496.210,2.489,2,2021-09-22T09:05:00,2021-09-22T09:10:00,2.48900,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,903.476,451.738,2.282,3,2021-09-22T09:10:00,2021-09-22T09:15:00,4.33310,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,928.459,464.230,2.193,4,2021-09-22T09:15:00,2021-09-22T09:20:00,4.34442,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1087.300,543.650,2.109,5,2021-09-22T09:20:00,2021-09-22T09:25:00,4.35487,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,937.218,468.609,2.031,6,2021-09-22T09:25:00,2021-09-22T09:30:00,4.36520,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1083.360,541.680,1.959,7,2021-09-22T09:30:00,2021-09-22T09:35:00,4.37520,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,987.842,493.921,1.894,8,2021-09-22T09:35:00,2021-09-22T09:40:00,4.38565,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,955.111,477.555,1.835,9,2021-09-22T09:40:00,2021-09-22T09:45:00,4.39536,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1035.550,517.776,4.438,10,2021-09-22T09:45:00,2021-09-22T09:50:00,7.06013,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
